In [81]:
def get_comment(code_str):
    code_template = """你现在是一个程序员助手. 程序员在写了代码之后忘记给代码注释. 你的任务是给程序员的代码进行合理恰当的中文注释。让用户能够快速的理解相关的代码。只输出注释后的代码。不输出其它任何多余的内容.
    Q:
    class StreamHandler(BaseCallbackHandler):
    def __init__(self, container, initial_text=""):
        self.container = container
        self.text = initial_text
    def on_llm_new_token(self, token: str, **kwargs) -> None:
        self.text += token
        self.container.markdown(self.text)
    A:
    ```python
    class StreamHandler(BaseCallbackHandler):
    \"\"\"
    StreamHandler 类用于处理从语言模型（LLM）接收的新标记并更新一个容器的显示内容。

    这个类继承自 BaseCallbackHandler，是一个回调处理器，专门用于处理和响应来自语言模型的新生成的标记。

    Attributes:
        container: 容器对象，用于在其中显示文本。
        text: 初始文本，用于在容器中开始显示。默认为空字符串。

    Methods:
        on_llm_new_token(token: str, **kwargs): 当从语言模型接收到新标记时调用。将新标记附加到现有文本并更新容器的显示。
    \"\"\"

    def __init__(self, container, initial_text=""):
        \"\"\"
        初始化 StreamHandler 实例。

        Args:
            container: 容器对象，用于显示来自语言模型的文本。
            initial_text: 可选；初始文本字符串，用于在容器中开始显示。默认为空字符串。
        \"\"\"
        self.container = container
        self.text = initial_text

    def on_llm_new_token(self, token: str, **kwargs) -> None:
        \"\"\"
        当从语言模型接收到新标记时被调用的方法。

        这个方法将接收到的标记附加到类实例的文本属性上，并更新容器中的显示内容。

        Args:
            token: 从语言模型接收到的新标记字符串。
            **kwargs: 可选参数，用于额外的功能或处理。
        \"\"\"
        self.text += token
        self.container.markdown(self.text)
        ```
    Q:
    """
    
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", code_template),
            ("human", "{input}")
        ]
    )
    llm = ChatOpenAI(
        openai_api_base="https://aiapi.xing-yun.cn/v1",
        openai_api_key="sk-RSAL5bknVmekLf005e714770B4Af431d821397F97d865cEb",
        model_name="gpt-3.5-turbo",
        temperature=0
    )
    # llm = Ollama(model="yi:34b-chat")
    chain = prompt | llm
    response = chain.invoke({"input": code_str})
    # print(f"Response is {response.content}")
    # rt = extract_code(response.content)
    # return rt
    if not response.content:
        return ""
    return response.content

In [1]:
import os
import shutil
from pprint import pprint 
import warnings
import re
warnings.filterwarnings("ignore")
from langchain.document_loaders import TextLoader
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import TextSplitter
from langchain.text_splitter import Language
from langchain.schema.output_parser import StrOutputParser
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.llms import Ollama
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from operator import itemgetter


In [11]:
def get_splitter(file):
    python_splitter = RecursiveCharacterTextSplitter.from_language(
        language=Language.PYTHON, chunk_size=2000, chunk_overlap=0
    )

    java_splitter = RecursiveCharacterTextSplitter.from_language(
        language=Language.JAVA, chunk_size=2000, chunk_overlap=0
    )
    cpp_splitter = RecursiveCharacterTextSplitter.from_language(
        language=Language.CPP, chunk_size=2000, chunk_overlap=0
    )
    Javascript_splitter = RecursiveCharacterTextSplitter.from_language(
        language=Language.JS, chunk_size=2000, chunk_overlap=0
    )
    match file.split('.'):
        case [_, 'py']:
            print("Python splitter")
            return python_splitter
        case [_, 'java']:
            print("Java splitter")
            return java_splitter
        case [_, 'cpp']:
            print("C++ splitter")
            return cpp_splitter
        case [_, 'js']:
            print("Javascript splitter")
            return Javascript_splitter
        case _:
            return None

In [12]:
splitter = get_splitter(file_path)
splitter

Python splitter


In [13]:
file_path = os.path.join("data", "RiseGPT", "app.py")
loader = TextLoader(file_path)
documents = loader.load()
texts = splitter.split_documents(documents)
len(texts)

4

In [ ]:
for  t in texts:
    print("--------------------------------------------------")
    print(t.page_content)

In [82]:
s_lst = [get_comment(t.page_content) for t in texts]

In [83]:
rt = "-----\n".join(s_lst)
print(rt)

```python
import streamlit as st
from langchain.callbacks.base import BaseCallbackHandler
from utils import *
import json
import os 
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import Chroma
from langchain.chains import LLMChain
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
)
from operator import itemgetter
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough

_template = """Answer the user questions based on the context. 
<context>
{context}
<context/>
"""
# Prompt


os.environ["OPENAI_API_BASE"] = "https://aiapi.xing-yun.cn/v1" 
os.environ["OPENAI_API_KEY"] = "sk-3e5wTBAl2iFDvQvW9b5693C90a97425eBf3b4bEa558eC66a"
st.title("RiseGPT")
with st.expander("ℹ️ 说明"):
    st.caption(
        "重庆市西南大学Rise实验室刘志明老师论文助手(第一次加载请稍等)"
    )

if "openai_model" not in st.session_state:
    st.sessi

In [84]:
print(s_lst[-1])

```python
with st.chat_message("assistant"):
    # 初始化一个空字符串，用于存储完整的回复内容
    full_response = ""
    # 调用 get_response 函数获取回复内容，并将其添加到 full_response 中
    response = get_response(question, docs)
    full_response += response
    # 将文档的来源信息拼接成一个字符串，并使用换行符分隔每个来源
    sources = "\n\n".join([f"📚 来源 {i + 1}: { remain_last( d.metadata['source'] ) } 第 {d.metadata['page']}页" for i, d in enumerate(docs)])
    # 创建一个空的 message_placeholder 对象
    message_placeholder = st.empty()
    # 在 message_placeholder 中显示来源信息
    message_placeholder.markdown(sources)
# 将完整的回复内容添加到会话状态的 messages 列表中
st.session_state.messages.append(
    {"role": "assistant", "content": full_response}
)
```


In [85]:
type(s_lst[0])

str

In [54]:
s_lst[-1].count("cpp")

1

In [86]:
s_lst[0]

'```python\nimport streamlit as st\nfrom langchain.callbacks.base import BaseCallbackHandler\nfrom utils import *\nimport json\nimport os \nfrom langchain.chat_models import ChatOpenAI\nfrom langchain.memory import ConversationBufferMemory\nfrom langchain.vectorstores import Chroma\nfrom langchain.chains import LLMChain\nfrom langchain.prompts import (\n    ChatPromptTemplate,\n    MessagesPlaceholder,\n)\nfrom operator import itemgetter\nfrom langchain.prompts import ChatPromptTemplate, MessagesPlaceholder\nfrom langchain.schema.runnable import RunnableLambda, RunnablePassthrough\n\n_template = """Answer the user questions based on the context. \n<context>\n{context}\n<context/>\n"""\n# Prompt\n\n\nos.environ["OPENAI_API_BASE"] = "https://aiapi.xing-yun.cn/v1" \nos.environ["OPENAI_API_KEY"] = "sk-3e5wTBAl2iFDvQvW9b5693C90a97425eBf3b4bEa558eC66a"\nst.title("RiseGPT")\nwith st.expander("ℹ️ 说明"):\n    st.caption(\n        "重庆市西南大学Rise实验室刘志明老师论文助手(第一次加载请稍等)"\n    )\n\nif "openai_model" no

In [87]:

def extract_code(text):
    pattern = r"```python(.*?)```"
    matches = re.findall(pattern, text, re.DOTALL)
    return matches 
    

In [88]:
text = s_lst[0]
# print(f"The str is {text}")
rt = extract_code(text)
print("\n".join([r for r in rt]))


import streamlit as st
from langchain.callbacks.base import BaseCallbackHandler
from utils import *
import json
import os 
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import Chroma
from langchain.chains import LLMChain
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
)
from operator import itemgetter
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough

_template = """Answer the user questions based on the context. 
<context>
{context}
<context/>
"""
# Prompt


os.environ["OPENAI_API_BASE"] = "https://aiapi.xing-yun.cn/v1" 
os.environ["OPENAI_API_KEY"] = "sk-3e5wTBAl2iFDvQvW9b5693C90a97425eBf3b4bEa558eC66a"
st.title("RiseGPT")
with st.expander("ℹ️ 说明"):
    st.caption(
        "重庆市西南大学Rise实验室刘志明老师论文助手(第一次加载请稍等)"
    )

if "openai_model" not in st.session_state:
    st.session_state[

In [93]:
from enum import Enum
class Language(Enum):
    PYTHON = "python"
    JAVA = "java"
    CPP = "cpp"
    JS = "js"

python = Language.PYTHON


In [94]:
print(python)

Language.PYTHON


In [95]:
print(python.value)

python


In [96]:
print(Language.CPP.value)

cpp
